In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


# Reading the Data files

In [2]:
data_raw = pd.read_csv('/kaggle/input/titanic/train.csv')
data_for_test = pd.read_csv('/kaggle/input/titanic/test.csv')

data_raw.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [3]:
data_for_test['Embarked']

0      Q
1      S
2      Q
3      S
4      S
      ..
413    S
414    C
415    S
416    S
417    C
Name: Embarked, Length: 418, dtype: object

# Defining a Function that will PREPARE THE DATA on demand

In [4]:
object_cols = list(data_raw.select_dtypes(include=np.object_))
num_cols = list(data_raw.select_dtypes(include=np.number))

print("object_cols\n", object_cols,)
print("\nnum_cols\n", num_cols)

data_raw['Embarked'].value_counts()

object_cols
 ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']

num_cols
 ['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']


S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [5]:
def prepare_data(df,y=True, get_df = True):
    object_cols = list(df.select_dtypes(include=np.object_))
    num_cols = list(df.select_dtypes(include=np.number))
    
    # copying the data to make changes only in copied data
    dfc = df.copy()
    
    # this column not necessary for training
    num_cols.remove('PassengerId')
    if 'Survived' in num_cols:
        num_cols.remove('Survived')
    
    object_cols.remove('Name')
    
    # filling the nan values
    dfc['Embarked'].fillna('S', inplace=True)
    dfc['Sex'].fillna('female', inplace=True)
    dfc['Name'].fillna('missing', inplace=True)
    
    for x in num_cols:
        mn = dfc[x].mean()
        dfc[x].fillna(mn , inplace=True)
        
    # making new columns
    dfc['family_mems'] = dfc['SibSp'] + dfc['Parch'] + 1 # 1 because counting self
    
    def decide_marital_status(val):
        if val==1:
            return 'Single'
        elif val==2:
            return 'Couple'
        elif val>2:
            return 'Family'
    
    def decide_age_group(a):
        if a<=5:
            return 'infant'
        elif 5<a<14:
            return 'child'
        elif 14<a<25:
            return 'youth'
        else:
            return 'adult'
    
    # new column -> marital status
    dfc['marital_status'] = dfc['family_mems'].apply(decide_marital_status)
    # new column -> age_group
    dfc['age_group'] = dfc['Age'].apply(decide_age_group)
    # new column -> Title
    titles = [some_name.split(',')[1].split('.')[0].strip() for some_name in df['Name']]
    if 'missing' in titles:
        titles.remove('missing')
    dfc['Title'] = titles
    dfc['Title'] = dfc['Title'].replace(
        ['Lady', 'the Countess', 'Countess', 'Capt', 
        'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 
        'Jonkheer', 'Dona', 'Ms', 'Mme', 'Mlle'], 
        'Rare')
    
    # Drop or not does not matter because we will be selecting columns
    # mentioned in *attr_for_onehot* and *attr_for_scaling*
    # Dropping columns assumed not necessary for training
    # dfc.drop(['PassengerId','Ticket','Cabin','SibSp','Parch','Name'], axis=1, inplace=True)
    
    attr_for_onehot = ['Sex','Embarked','age_group','marital_status', 'Title']
    attr_for_scaling = ['Fare','Age', 'family_mems']
    
    from sklearn.preprocessing import OneHotEncoder, StandardScaler
    one_hot = OneHotEncoder(sparse=False)
    scaler = StandardScaler()
    one_hot_tr = one_hot.fit_transform(dfc[attr_for_onehot])
    scale_tr = scaler.fit_transform(dfc[attr_for_scaling])
    
    data_prepared = np.concatenate((one_hot_tr, scale_tr), axis=1)
    
    if get_df == True:
        features = list(one_hot.get_feature_names_out()) + attr_for_scaling
        data_prepared_df = pd.DataFrame(data_prepared,
                                       columns=features,
                                       index = dfc.index)
        if y==False:
            return data_prepared_df
        return data_prepared_df, dfc['Survived']
    
    if y==False:
            return data_prepared
    return data_prepared, dfc['Survived']

In [6]:
data_train_prepared, label = prepare_data(data_raw, get_df=True)

In [7]:
data_train_prepared.head(10)

,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,age_group_adult,age_group_child,age_group_infant,age_group_youth,marital_status_Couple,marital_status_Family,marital_status_Single,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Rare,Fare,Age,family_mems
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.502445,-0.592481,0.059160
1,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.786845,0.638789,0.059160
2,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,-0.488854,-0.284663,-0.560975
3,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.420730,0.407926,0.059160
4,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,-0.486337,0.407926,-0.560975
5,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,-0.478116,0.000000,-0.560975
6,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.395814,1.870059,-0.560975
7,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.224083,-2.131568,1.919564
8,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.424256,-0.207709,0.679295
9,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.042956,-1.208115,0.059160


# I inferred from previous version that RandomForestClassifier seems good for Predicitons

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
from sklearn.metrics import precision_score, recall_score

precisions = []
recalls = []
scores = []

for n in range(20):
    X, y = shuffle(data_train_prepared, label)
    forest_clf = RandomForestClassifier()
    forest_clf.fit(X, y)
    y_pred = forest_clf.predict(X)
    precisions.append(precision_score(y,y_pred))
    recalls.append(recall_score(y,y_pred))
    scores.append(forest_clf.score(X, y))
    
print(sum(precisions)/len(precisions))
print(sum(recalls)/len(recalls))
print(sum(scores)/len(scores))

0.9876727137427679
0.9710526315789471
0.9842312008978679


# Making predictions for Test data

In [9]:
test_prepared = prepare_data(data_for_test, y=False, get_df=True)

# for col in data_train_prepared.columns:
#     if col not in test_prepared.columns:
#         print(col)
test_pred = forest_clf.predict(test_prepared)

submission = pd.DataFrame({
    'PassengerId': data_for_test['PassengerId'],
    'Survived': test_pred
})

submission.to_csv('submission.csv', index=False)

